In [1]:
!pip install sodapy

    100% |████████████████████████████████| 829kB 744kB/s ta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/davidplewis/Library/Caches/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built future
distributed 1.21.8 requires msgpack, which is not installed.


In [3]:
# from https://github.com/socrata/dev.socrata.com/blob/39c6581986466edb5e7f72f5beea5ce69238f8de/snippets/pandas.py

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# First 50000 results, returned as JSON from API 
# Connverted to Python list of dictionaries by sodapy.
# Column names converted to snake case, special chars removed
# Dates and location formatted
results = client.get("r5kz-chrr", limit=50000)

# Convert to pandas DataFrame
licenses = pd.DataFrame.from_records(results)

In [4]:
# download remaining (limit 50000 / call)
start = 50000
while results:
    print(start)
    results = client.get("r5kz-chrr", limit=50000, offset=start)
    licenses = licenses.append(pd.DataFrame.from_records(results))
    start += 50000


50000
100000
150000
200000
250000
300000
350000
400000
450000
500000


/Users/davidplewis/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


550000
600000
650000
700000
750000
800000
850000
900000
950000


In [12]:
# Query data to get an idea of what information is present
licenses.head()

,account_number,address,application_created_date,application_requirements_complete,application_type,business_activity,business_activity_id,city,conditional_approval,date_issued,...,longitude,payment_date,police_district,precinct,site_number,ssa,state,ward,ward_precinct,zip_code
0,51755,11243 CHESAPEAKE PLAC 1ST,NaN,2000-06-16T00:00:00,RENEW,NaN,NaN,WESTCHESTER,N,2009-08-24T00:00:00,...,NaN,2009-08-21T00:00:00,NaN,NaN,1,NaN,IL,NaN,NaN,60154
1,51944,5410 NEWPORT DR 40,NaN,2000-06-20T00:00:00,ISSUE,NaN,NaN,ROLLING MEADOWS,N,2006-04-12T00:00:00,...,NaN,NaN,NaN,NaN,1,NaN,IL,NaN,NaN,60008
2,52896,17021 S MAGNOLIA DR 1ST,NaN,2000-06-20T00:00:00,ISSUE,NaN,NaN,HAZEL CREST,N,2006-04-12T00:00:00,...,NaN,NaN,NaN,NaN,1,NaN,IL,NaN,NaN,60429
3,54389,450 SKOKIE BLVD 904,NaN,2000-06-20T00:00:00,ISSUE,NaN,NaN,NORTHBROOK,N,2006-04-12T00:00:00,...,NaN,NaN,NaN,NaN,1,NaN,IL,NaN,NaN,60062
4,55668,261 N WEST AVE 1ST,NaN,2000-06-20T00:00:00,ISSUE,NaN,NaN,ELMHURST,N,2006-04-12T00:00:00,...,NaN,NaN,NaN,NaN,1,NaN,IL,NaN,NaN,60126


In [7]:
type(licenses.iloc[40].application_requirements_complete)


str

In [8]:
# Drop duplicates
licenses.drop_duplicates("id", inplace=True)

In [7]:
# Drop rows with missing license term start date?
# Drop rows with missing license term expiration date?
# drop application types C_CAPA and C_SBA?business_licenses.to_csv("../../DATA/business_licenses")

In [9]:
import os.path
root_path = os.path.dirname(os.getcwd())

# Save result
licenses.to_csv(os.path.join(root_path, "DATA/business_licenses.csv"), index=False)